# Proyecto Sprint 11: Aprendizaje Automático


**Descripción del proyecto**

Trabajas en la compañía de extracción de petróleo OilyGiant. Tu tarea es encontrar los mejores lugares donde abrir 200 pozos nuevos de petróleo.

Para completar esta tarea, tendrás que realizar los siguientes pasos:

- Leer los archivos con los parámetros recogidos de pozos petrolíferos en la región seleccionada: calidad de crudo y volumen de reservas.
- Crear un modelo para predecir el volumen de reservas en pozos nuevos.
- Elegir los pozos petrolíferos que tienen los valores estimados más altos.
- Elegir la región con el beneficio total más alto para los pozos petrolíferos seleccionados.

Tienes datos sobre muestras de crudo de tres regiones. Ya se conocen los parámetros de cada pozo petrolero de la región. Crea un modelo que ayude a elegir la región con el mayor margen de beneficio. Analiza los beneficios y riesgos potenciales utilizando la técnica bootstrapping.

- Solo se debe usar la regresión lineal para el entrenamiento del modelo.
- Al explorar la región, se lleva a cabo un estudio de 500 puntos con la selección de los mejores 200 puntos para el cálculo del beneficio.
- El presupuesto para el desarrollo de 200 pozos petroleros es de 100 millones de dólares.
- Un barril de materias primas genera 4.5 USD de ingresos. El ingreso de una unidad de producto es de 4500 dólares (el volumen de reservas está expresado en miles de barriles).
- Después de la evaluación de riesgo, mantén solo las regiones con riesgo de pérdidas inferior al 2.5%. De las que se ajustan a los criterios, se debe seleccionar la región con el beneficio promedio más alto.

## Preparación de los datos

In [1]:
# importar las librerías
import pandas as pd
import numpy as np
from numpy.random import RandomState
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.metrics import mean_absolute_error



In [2]:
# crea los dataframe

geo_df_0 = pd.read_csv('/datasets/geo_data_0.csv')
#geo_df_0

geo_df_1 = pd.read_csv('/datasets/geo_data_1.csv')
#geo_df_1

geo_df_2 = pd.read_csv('/datasets/geo_data_2.csv')
geo_df_2


,id,f0,f1,f2,product
0,fwXo0,-1.146987,0.963328,-0.828965,27.758673
1,WJtFt,0.262778,0.269839,-2.530187,56.069697
2,ovLUW,0.194587,0.289035,-5.586433,62.871910
3,q6cA6,2.236060,-0.553760,0.930038,114.572842
4,WPMUX,-0.515993,1.716266,5.899011,149.600746
...,...,...,...,...,...
99995,4GxBu,-1.777037,1.125220,6.263374,172.327046
99996,YKFjq,-1.261523,-0.894828,2.524545,138.748846
99997,tKPY3,-1.199934,-2.957637,5.219411,157.080080
99998,nmxp2,-2.419896,2.417221,-5.548444,51.795253


In [3]:
# eliminar la columna 'id', de los 3 dataframes, irrelevante para el entrenamiento del modelo
#geo_df_0 = geo_df_0.drop('id', axis = 1)
#geo_df_1 = geo_df_1.drop('id', axis = 1)
#geo_df_2 = geo_df_2.drop('id', axis = 1)

#geo_df_0

## Entrenamiento del modelo

In [4]:
# función para dividir los datos y entrenar el modelo

numeric_columns = ['f0', 'f1', 'f2']

def split_and_train_model(df, numeric_columns, test_size=0.25, random_state=12345):
   
    # Separar en características (features) y objetivo (target)
    target = df['product']
    features = df.drop(['product', 'id'], axis=1)
    
    # Dividir en entrenamiento y validación
    features_train, features_valid, target_train, target_valid = train_test_split(
        features, target, test_size=test_size, random_state=random_state)
    
    # Escalar las características numéricas
    scaler = StandardScaler()
    scaler.fit(features_train[numeric_columns])
    
    features_train[numeric_columns] = scaler.transform(features_train[numeric_columns])
    features_valid[numeric_columns] = scaler.transform(features_valid[numeric_columns])

    # Inicializar el modelo
    model = LinearRegression()
    
    # Entrenar el modelo
    model.fit(features_train, target_train)
    
    # Realizar las predicciones
    predicted_valid = model.predict(features_valid)
    
    # Calcular el MSE y RMSE
    mse = mean_squared_error(target_valid, predicted_valid)
    rmse = mse ** 0.5
    
    results = pd.DataFrame({
        'target': target_valid,
        'predicted': predicted_valid
    })
    
    return results, rmse

### Primera región

In [5]:
# aplica la función a la primera región
results, rmse = split_and_train_model(geo_df_0, numeric_columns)
region_0_data = results
print(region_0_data)
mean_volume_predicted_0 = results['predicted'].mean()
mean_volume_0 = results['target'].mean()
print('Volumen medio de reservas predicho', mean_volume_predicted_0)
print('Volumen medio de reservas real', mean_volume_0)
print("RMSE =", rmse)



           target   predicted
71751   10.038645   95.894952
80493  114.551489   77.572583
2655   132.603635   77.892640
53233  169.072125   90.175134
91141  122.325180   70.510088
...           ...         ...
12581  170.116726  103.037104
18456   93.632175   85.403255
73035  127.352259   61.509833
63834   99.782700  118.180397
43558  177.821022  118.169392

[25000 rows x 2 columns]
Volumen medio de reservas predicho 92.59256778438035
Volumen medio de reservas real 92.07859674082927
RMSE = 37.5794217150813


/tmp/ipykernel_161/2666597112.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[numeric_columns] = scaler.transform(features_train[numeric_columns])
/opt/conda/envs/python3/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/envs/python3/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

### Segunda región

In [6]:
# aplica la función a la segunda región
results, rmse = split_and_train_model(geo_df_1, numeric_columns)
region_1_data = results
print(region_1_data)
mean_volume_predicted_1 = results['predicted'].mean()
mean_volume_1 = results['target'].mean()
print('Volumen medio de reservas predicho', mean_volume_predicted_1)
print('Volumen medio de reservas real', mean_volume_1)
print("RMSE =", rmse)

           target   predicted
71751   80.859783   82.663314
80493   53.906522   54.431786
2655    30.132364   29.748760
53233   53.906522   53.552133
91141    0.000000    1.243856
...           ...         ...
12581  137.945408  136.869211
18456  110.992147  110.693465
73035  137.945408  137.879341
63834   84.038886   83.761966
43558   53.906522   53.958466

[25000 rows x 2 columns]
Volumen medio de reservas predicho 68.728546895446
Volumen medio de reservas real 68.72313602435997
RMSE = 0.893099286775617


/tmp/ipykernel_161/2666597112.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[numeric_columns] = scaler.transform(features_train[numeric_columns])
/opt/conda/envs/python3/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/envs/python3/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

### Tercera región

In [7]:
# aplica la función a la tercera región
results, rmse = split_and_train_model(geo_df_2, numeric_columns)
region_2_data = results
print(region_2_data)
mean_volume_predicted_2 = results['predicted'].mean()
mean_volume_2 = results['target'].mean()
print('Volumen medio de reservas predicho', mean_volume_predicted_2)
print('Volumen medio de reservas real', mean_volume_2)
print("RMSE =", rmse)

           target   predicted
71751   61.212375   93.599633
80493   41.850118   75.105159
2655    57.776581   90.066809
53233  100.053761  105.162375
91141  109.897122  115.303310
...           ...         ...
12581   28.492402   78.765887
18456   21.431303   95.603394
73035  125.487229   99.407281
63834   99.422903   77.779912
43558  127.445075  129.032417

[25000 rows x 2 columns]
Volumen medio de reservas predicho 94.96504596800489
Volumen medio de reservas real 94.88423280885438
RMSE = 40.02970873393434


/tmp/ipykernel_161/2666597112.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features_train[numeric_columns] = scaler.transform(features_train[numeric_columns])
/opt/conda/envs/python3/lib/python3.9/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/opt/conda/envs/python3/lib/python3.9/site-packages/pandas/core/indexing.py:692: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas

Se observa que el modelo de la segunda región tiene un error cuadrado de 0.893 miles de barriles, mientras que la primera y tercer región tienen un error cuadrado de 37.579 y 40.029.

## Preparación para el cálculo de ganancias

In [8]:
# crea las variables con las condiciones del proyecto 
budget = 100000000
well_count = 200
earnings_per_unit = 4500
earnings_per_barrel = earnings_per_unit/1000
minimum_return_per_well = 500000


In [9]:
# calcular el volumen medio de reservas predicho de cada región

threshold = minimum_return_per_well / earnings_per_unit

# Comparar con el umbral
print(f"Volumen medio de reservas en la región 1: {mean_volume_predicted_0} miles de barriles")
print(f"Volumen medio de reservas en la región 2: {mean_volume_predicted_1} miles de barriles")
print(f"Volumen medio de reservas en la región 3: {mean_volume_predicted_2} miles de barriles")

# Compara con el umbral
# Comparación directa de los volúmenes medios con el umbral
print(f"Región 0 {'cumple' if mean_volume_0 >= threshold else 'NO cumple'} con el umbral de ingresos de {minimum_return_per_well} USD")
print(f"Región 1 {'cumple' if mean_volume_1 >= threshold else 'NO cumple'} con el umbral de ingresos de {minimum_return_per_well} USD")
print(f"Región 2 {'cumple' if mean_volume_2 >= threshold else 'NO cumple'} con el umbral de ingresos de {minimum_return_per_well} USD")

Volumen medio de reservas en la región 1: 92.59256778438035 miles de barriles
Volumen medio de reservas en la región 2: 68.728546895446 miles de barriles
Volumen medio de reservas en la región 3: 94.96504596800489 miles de barriles
Región 0 NO cumple con el umbral de ingresos de 500000 USD
Región 1 NO cumple con el umbral de ingresos de 500000 USD
Región 2 NO cumple con el umbral de ingresos de 500000 USD


Ninguna región cumple con el umbral mínimo de ingresos, es decir, de 111.1 unidades 

## Cálculo de ganancia de pozos seleccionados

In [10]:
def earning_calc(df):
    # Ordenar por la predicción de reservas
    best_wells = df.sort_values(by='predicted', ascending=False).head(200)
    
    # Calcular la suma de las reservas de los 200 pozos
    total_volume = best_wells['predicted'].sum()
    
    # Calcular la ganancia
    earnings = total_volume * earnings_per_unit 
    
    # Verificar si se ajusta al presupuesto
    if earnings >= budget:
        print(f"La ganancia potencial es de ${earnings}. Supera el presupuesto.")
    else:
        print(f"La ganancia potencial es de ${earnings}. NO supera el presupuesto.")
    
    return earnings

# aplicar a las regiones
earnings_region_0 = earning_calc(region_0_data)
earnings_region_1 = earning_calc(region_1_data)
earnings_region_2 = earning_calc(region_2_data)

# Conclusión
if earnings_region_0 > earnings_region_1 and earnings_region_0 > earnings_region_2:
    print("La región 0 es la mejor opción para desarrollar los pozos.")
elif earnings_region_1 > earnings_region_0 and earnings_region_1 > earnings_region_2:
    print("La región 1 es la mejor opción para desarrollar los pozos.")
else:
    print("La región 2 es la mejor opción para desarrollar los pozos.")

La ganancia potencial es de $139960488.77465132. Supera el presupuesto.
La ganancia potencial es de $124857120.51973544. Supera el presupuesto.
La ganancia potencial es de $133217543.96243258. Supera el presupuesto.
La región 0 es la mejor opción para desarrollar los pozos.


## Cálculo de riesgos y ganancias para cada región

Se aplica la técnica de remuestreo de bootstrapping para estimar las ganancias, con reemplazo, para calcular el beneficio promedio, el intervalo de confianza y riesgo de pérdidas.

In [17]:
state = RandomState(12345)

def bootstrap(region_data, size=1000):
    # Realizar bootstrapping
    earnings_bootstrap = []
    
    for i in range(size):
        # Realizar un muestreo aleatorio con reemplazo de los 200 mejores pozos
        sample = region_data.sample(n=200, replace=True, random_state = state)
        
        # Calcular la ganancia de esta muestra
        sample_volume = sample['predicted'].sum()
        sample_earning = sample_volume * earnings_per_unit
        earnings_bootstrap.append(sample_earning)
    
    # Calcular el beneficio promedio y el intervalo de confianza
    mean_earning = np.mean(earnings_bootstrap)
    interval = np.percentile(earnings_bootstrap, [2.5, 97.5])  # Intervalo de confianza del 95%
    
    # Calcular riesgo de pérdidas
    loss = len([x for x in earnings_bootstrap if x < budget]) / size * 100
    
    return mean_earning, interval, loss

# Aplicar bootstrapping a cada región
mean_earnings_0, interval_0, loss_0 = bootstrap(region_0_data)
mean_earnings_1, interval_1, loss_1 = bootstrap(region_1_data)
mean_earnings_2, interval_2, loss_2 = bootstrap(region_2_data)

print(f"Región 0: Ganancia promedio: {mean_earnings_0:.2f}, IC 95%: {interval_0}, Riesgo de pérdidas: {loss_0}%")
print(f"Región 1: Ganancia promedio: {mean_earnings_1:.2f}, IC 95%: {interval_1}, Riesgo de pérdidas: {loss_1}%")
print(f"Región 2: Ganancia promedio: {mean_earnings_2:.2f}, IC 95%: {interval_2}, Riesgo de pérdidas: {loss_2}%")


Región 0: Ganancia promedio: 83374370.06, IC 95%: [80784765.2121128  86173543.61871937], Riesgo de pérdidas: 100.0%
Región 1: Ganancia promedio: 61776643.86, IC 95%: [56589593.15879872 67713594.84185767], Riesgo de pérdidas: 100.0%
Región 2: Ganancia promedio: 85519650.69, IC 95%: [83022793.93277714 88024683.62345989], Riesgo de pérdidas: 100.0%


## Conclusión

Aplicando la técnica de bootstrapping, ninguna región sería viable para realizar el proyecto. La región 2 tiene mayores posibilidades de ingreso, pero ninguna de alcanzar o superar el presupuesto.